In [5]:
import pickle

In [11]:
import pickle
import sys
import nltk
import torch.nn as nn
import numpy as np
nltk.download('punkt')
from gensim.models import Word2Vec
sys.path.insert(0, '..')
import torch

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def save_lists_to_file(path):
    # Open the pickle file in binary write mode
    with open(path, 'rb') as f:
    # Load each list from the file
        NE = pickle.load(f)
        EEG_segments = pickle.load(f)
        Classes = pickle.load(f)

    return NE, EEG_segments, Classes

In [18]:
def create_word_label_embeddings(Word_Labels_List, word_embedding_dim=50):
    """
    This function takes in the list of words associated with EEG segments and returns the word embeddings for each word

    :param Word_Labels_List: List of the textual data associated with EEG segments
    :return Embedded_Word_labels: List of each words embeddings
    :return word_embeddings: Dictionary of word embeddings
    """
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))

    Embedded_Word_labels = []
    for word in Word_Labels_List:
        Embedded_Word_labels.append(word_embeddings[word])

    return Embedded_Word_labels, word_embeddings

In [14]:
train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [15]:
train_NE, train_EEG_segments, train_Classes = save_lists_to_file(train_path)
test_NE, test_EEG_segments, test_Classes = save_lists_to_file(test_path)

In [23]:
len(train_EEG_segments[0][0])

2

In [25]:
len(train_NE[0])

2

In [36]:
len(train_EEG_segments[0])

61

In [33]:

for i in range(len(train_NE)):
    Named_Entity = train_NE[i]
    Named_Entity_EEG_Segments = train_EEG_segments[i]
    for j in range(len(Named_Entity)):
        word = Named_Entity[j]
        print(word)




Henry
Ford
Harry
Bennett
Edsel
Eleanor
Henry
Ford
II
Aldous
Leonard
Huxley
United
States
Huxley
family
Huxley
Godalming
Surrey
England
Leonard
Huxley
Julia
Arnold
Thomas
Henry
Huxley
Darwin
Julian
Huxley
John
Wells
Wells
David
Frost
Millicent
Martin
BBC
Nancy
Cartwright
Homer
Marge
Simpson
Lisa
Maggie
Roy
Stanley
Geiger
United
States
Marine
Corps
Marine
Geiger
Middleburg
Florida
Florida
State
Normal
and
Industrial
College
and
Stetson
University
Marine
Corps
School
Port
Royal
South
Carolina
Detroit
III
Amphibious
Corps
Okinawa
General
Tenth
Army
Joseph
Stilwell
Culbert
Levy
Olson
Fillmore
Utah
Daniel
Olson
of
United
Secularists
of
America
Los
Angeles
Glendale
Prescott
Sheldon
Bush
Columbus
Ohio
New
York
City
Connecticut
Brown
Brothers
Harriman
Samuel
Prescott
Bush
Flora
Sheldon
Dorothy
Walker
George
Herbert
Walker
Prescott
Bush
Kennebunkport
Maine
Isabel
Stillman
Rockefeller
Percy
Rockefeller
Hope
Lincoln
Mary
Keck
Elizabeth
Trotter
Martha
Pittman
Ruth
Lionberger
Nancy
Walker
Knight
Woo

In [16]:

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [19]:
#embeddings
Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)


Number of word embeddings: 5860
